In [67]:
import matplotlib.pyplot as plt
%matplotlib inline
import subprocess
import shlex
import numpy as np
import pandas as pd

import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(repo_root)

from src.utils.file_handlers import group_dataset_files, read_dataset_files
from src.utils.operation_helpers import run_models, split_games
from src.utils.c_operation_helpers import  run_simulation
from src.utils.file_handlers import process_directory_real_data

### Example

Python

In [68]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

data, pi_values = read_dataset_files(grouped['00103'], data_dir, is_synthetic=False)

# train, test = train_test_split(data, train_size=.8)
train, test = split_games(data, .8)

df = run_models(train, test, pi_values)
print(df)


         model  log-likelihoods  leadership-log-likelihood
0        HO_BT        -2.744749                  -0.832101
1       HOL_BT        -2.742304                  -0.823259
2          BIN        -2.792092                  -0.843856
3         BINL        -2.789306                  -0.837696
4  Spring_Rank        -2.887182                  -0.849661
5    Page_Rank        -3.827274                  -1.095218
6   Point_Wise        -2.922654                  -0.844673


C 

In [69]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

edge_file = grouped['00103']['edges']
node_file = grouped['00103']['nodes']

edge_path = os.path.join(data_dir, edge_file)
node_path = os.path.join(data_dir, node_file)

print(edge_path)
print(node_path)

results = run_simulation(node_path, edge_path, is_synthetic=0)
print(results)

/home/jackyeung99/senior_thesis/higher_order_ranking/datasets/Real_Data/00103_edges.txt
/home/jackyeung99/senior_thesis/higher_order_ranking/datasets/Real_Data/00103_nodes.txt
  model  av_error    spearman     kendall     prior   HO_Like   HOL_Like  \
0    HO  0.343786    0.129006    0.076697   -1.5618  -2.82652   -0.83498   
1   HOL  0.344067  -0.0413226  -0.0320306  -1.49698  -2.93079  -0.845838   
2   BIN  0.351847    0.130627   0.0743462    -1.595  -2.87615   -0.84819   
3  BINL  0.359405  -0.0248399  -0.0202762   -1.5568  -3.05076  -0.878658   

  iterations  
0         10  
1         10  
2          9  
3         10  


### Full Results

In [70]:
EXPERIMENT_PATH = os.path.join(repo_root, 'exp', 'ex04')
os.chdir(EXPERIMENT_PATH)

In [71]:
print(os.getcwd())

/home/jackyeung99/senior_thesis/higher_order_ranking/exp/ex04


In [72]:
process_directory_real_data(EXPERIMENT_PATH)

In [73]:


def process_results():
    # Read the summary CSV files
    log_likelihood_df = pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()
    leadership_log_likelihood_df = pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'leadership_log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()

    filtered_log_like = log_likelihood_df.drop(columns=['epoch'])
    filtered_leader_like = leadership_log_likelihood_df.drop(columns=['epoch'])
    
    log_like = subtract_columns(filtered_log_like, 1)
    leadership_log = subtract_columns(filtered_leader_like, 2)

    return log_like, leadership_log

def subtract_columns(df, compared_col):
    columns = df.columns.tolist()
    base_column = columns[compared_col] 
    print(base_column)
    for col in columns[1:]:
        
        df[col] = df[col] - df[base_column] 
    
    return df


def visualize_df(df, title):
    print(f"\n{title}\n")
    styled_df = df.style.set_table_styles(
    ).set_properties(**{
        'background-color': 'LightGray',
        'color': 'black',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'text-align': 'left'
    })
    

    # print(df.to_latex(escape=True))
    display(styled_df)
   

In [74]:


unique_datasets = pd.read_csv(os.path.join(os.getcwd(), 'results', 'log_likelihood_summary.csv'))['dataset'].unique()
dataset_info = pd.read_csv(os.path.join(repo_root, 'datasets', 'dataset_info.csv'))
filtered_dataset_info = dataset_info[dataset_info['Dataset_ID'].isin(unique_datasets)].set_index('Dataset_ID')

# print(dataset_info)
# domain_order = ["Election", "Sport", "Preferences", "Other"]
# filtered_dataset_info['Domain'] = pd.Categorical(filtered_dataset_info['Domain'], categories=domain_order, ordered=True)

filtered_dataset_info = filtered_dataset_info.sort_values(by=['K1', 'K2', 'R', 'Name'])
visualize_df(filtered_dataset_info, 'Datasets Tested')


Datasets Tested



,Name,N,M,R,K1,K2,K_avg
Dataset_ID,,,,,,,
28,APA_election,5,203896,40779.200000,1,5,3.637000
9,AGH_course_selection,9,299,33.222000,7,9,7.977000
14,sushi,102,10000,98.039000,10,10,10.000000


In [75]:
log_like, leadership_log = process_results()

def rename_df(df):
    merged_df = df.merge(dataset_info, left_on='dataset', right_on='Dataset_ID').drop(columns=['Dataset_ID','dataset'])
    # merged_df = merged_df.rename(columns={'Name': 'dataset_name'}).set_index('dataset_name')

    domain_order = ["Election", "Sport", "Preferences", "Other"]
    # merged_df['Domain'] = pd.Categorical(merged_df['Domain'], categories=domain_order, ordered=True)
    merged_df = merged_df.sort_values(by=['K1', 'K2', 'Name'])
    merged_df = merged_df[[ 'Name', 'HO_BT','HOL_BT', 'BIN', 'BINL', 'Spring_Rank', 'Page_Rank', 'Point_Wise']]

    return merged_df

HO_BT
HOL_BT


In [76]:

log_like = rename_df(log_like)
visualize_df(log_like, 'log likelihood')


log likelihood



,Name,HO_BT,HOL_BT,BIN,BINL,Spring_Rank,Page_Rank,Point_Wise
2,APA_election,0.000000,-8.030349,-8.020390,-8.029644,-30.488771,-8.025190,-8.022637
0,AGH_course_selection,0.000000,-69.569586,-37.792514,-69.807542,-77.421993,-54.622632,-44.238760
1,sushi,0.000000,-84.002455,-81.622219,-85.290577,-82.246758,-88.808490,-83.641189


In [77]:
leadership_log = rename_df(leadership_log)
visualize_df(leadership_log, 'Leadership log likelihood')


Leadership log likelihood



,Name,HO_BT,HOL_BT,BIN,BINL,Spring_Rank,Page_Rank,Point_Wise
2,APA_election,-0.003044,0.000000,-1.133486,-1.131509,-4.389327,-1.132327,-1.132335
0,AGH_course_selection,-0.971300,0.000000,-0.896640,-0.015293,-1.386154,-2.876898,-1.687371
1,sushi,-0.099966,0.000000,-2.082928,-2.025895,-2.127639,-2.306420,-2.194517


In [78]:
log_likelihood_df = (
    pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'log_likelihood_summary.csv'))
    .groupby(by=['dataset'])
    .mean()
    .reset_index()
    .drop(columns=['epoch'])
)

log_likelihood_df.style.highlight_max(axis=1, color='grey', subset=log_likelihood_df.columns.difference(['dataset']))

,dataset,HO_BT,HOL_BT,BIN,BINL,Spring_Rank,Page_Rank,Point_Wise
0,9,-38.109360,-69.569586,-37.792514,-69.807542,-77.421993,-54.622632,-44.238760
1,14,-81.946867,-84.002455,-81.622219,-85.290577,-82.246758,-88.808490,-83.641189
2,28,-8.022434,-8.030349,-8.020390,-8.029644,-30.488771,-8.025190,-8.022637


In [79]:
log_likelihood_df = (
    pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'leadership_log_likelihood_summary.csv'))
    .groupby(by=['dataset'])
    .mean()
    .reset_index()
    .drop(columns=['epoch'])
)

log_likelihood_df.style.highlight_max(axis=1, color='grey', subset=log_likelihood_df.columns.difference(['dataset']))

,dataset,HO_BT,HOL_BT,BIN,BINL,Spring_Rank,Page_Rank,Point_Wise
0,9,-0.986864,-0.015564,-0.896640,-0.015293,-1.386154,-2.876898,-1.687371
1,14,-2.120922,-2.020955,-2.082928,-2.025895,-2.127639,-2.306420,-2.194517
2,28,-1.134479,-1.131435,-1.133486,-1.131509,-4.389327,-1.132327,-1.132335
